<a href="https://colab.research.google.com/github/mahluwal/Tensorflow/blob/master/DataScienceProject7RestaurantRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()



Saving zomato.csv to zomato.csv


In [4]:
zomato = pd.read_csv("/content/zomato.csv")
zomato.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [5]:
zomato.describe()

,votes
count,51717.000000
mean,283.697527
std,803.838853
min,0.000000
25%,7.000000
50%,41.000000
75%,198.000000
max,16832.000000


In [6]:
zomato.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost(for two people)', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)'],
      dtype='object')

In [9]:
# Let's delete unnecessay columns
# here I will drop the column "dish_liked", "phone", "url"
zomato2 = zomato.drop(['url','dish_liked','phone'], axis=1)

In [10]:
print(zomato2.duplicated().sum())

43


In [11]:
# now let's drop the duplicate values
zomato2.drop_duplicates(inplace=True)

In [12]:
# Now let's have a look at the null values in the dataset
print(zomato2.isnull().sum())

address                           0
name                              0
online_order                      0
book_table                        0
rate                           7767
votes                             0
location                         21
rest_type                       227
cuisines                         45
approx_cost(for two people)     345
reviews_list                      0
menu_item                         0
listed_in(type)                   0
listed_in(city)                   0
dtype: int64


In [13]:
# let's drop all the null values
zomato2.dropna(how='any', inplace=True)

In [14]:
# Now I will change the names of some columns to make the columns easier to use in the process
zomato = zomato2.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})

In [15]:
#Some Transformations
zomato['cost'] = zomato['cost'].astype(str) #Changing the cost to string
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
zomato['cost'] = zomato['cost'].astype(float)

In [16]:
#Removing '/5' from Rates
zomato = zomato.loc[zomato.rate !='NEW']
zomato = zomato.loc[zomato.rate !='-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '') if type(x) == np.str else x
zomato.rate = zomato.rate.apply(remove_slash).str.strip().astype('float')

In [17]:
# Adjust the column names
zomato.name = zomato.name.apply(lambda x:x.title())
zomato.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomato.book_table.replace(('Yes','No'),(True, False),inplace=True)

In [18]:
## Computing Mean Rating
restaurants = list(zomato['name'].unique())
zomato['Mean Rating'] = 0

for i in range(len(restaurants)):
    zomato['Mean Rating'][zomato['name'] == restaurants[i]] = zomato['rate'][zomato['name'] == restaurants[i]].mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
zomato[['Mean Rating']] = scaler.fit_transform(zomato[['Mean Rating']]).round(2)

In [20]:
## Lower Casing
zomato["reviews_list"] = zomato["reviews_list"].str.lower()

In [21]:
# now I will define a function to remove punctuation from the reviews
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_punctuation(text))

In [23]:
# Now let's remove the stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])
zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_stopwords(text))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
# Now let's remove the urls from the reviews
import re
def removeurls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: removeurls(text))

zomato[['reviews_list', 'cuisines']].sample(10)

,reviews_list,cuisines
31609,rated 30 ratedn ordered meals taste ok phulka ...,"North Indian, Chinese"
25785,rated 30 ratedn visited place yesterday new ye...,"Kerala, Seafood, Chinese, Biryani, Juices"
21019,rated 40 ratedn ordered boneless chicken birya...,"Andhra, North Indian, Biryani"
684,rated 40 ratedn ordered veg cutlet sandwich pe...,Fast Food
31931,rated 40 ratedn place beer lover sure service ...,"Finger Food, North Indian, Italian"
6985,rated 30 ratedn ordered garlic bread place wed...,"North Indian, Chinese, BBQ"
29764,rated 40 ratedn definitely 45 placethe ambienc...,"Finger Food, North Indian, Continental"
27062,rated 45 ratedn crowdy sunday night invites tr...,"Cafe, Continental, Beverages"
19571,rated 20 ratedn nice place north indian food m...,"Kerala, Chinese, Arabian, Biryani"
28105,rated 50 ratedn extremely delicious mouth melt...,"Bakery, Desserts"


In [25]:
# let's process the names of reataurants
restaurant_names = list(zomato['name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]

In [26]:
# Now let's drop the unnecessary columns
zomato=zomato.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)

In [27]:
# Now I will randomly sample the data
df_percent = zomato.sample(frac=0.5)

df_percent.set_index('name', inplace=True)
indices = pd.Series(df_percent.index)

In [28]:
# Now Let's build the restaurant recommendation system
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#Now I will perform tf-idf vectorization on the dataset
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [29]:
# Now the final step is to create a function to recommend restaurants
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top restaurants
    recommendations = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommendations.append(list(df_percent.index)[each])
    
    # Creating the new data set to show similar restaurants
    df = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommendations:
        df = df.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))

    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df = df.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df = df.sort_values(by='Mean Rating', ascending=False).head(10)
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df)), name))   
    return df
recommend('Keto Kitchen')

TOP 4 RESTAURANTS LIKE Keto Kitchen WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Brownie Heaven,"Desserts, Fast Food",3.96,300.0
Desi Dhaba,"North Indian, Chinese",3.19,400.0
Desi Dhaba,"North Indian, Chinese, Rolls",3.19,300.0
Keto Kitchen,Healthy Food,3.16,500.0


In [30]:
recommend('Pai Vihar')

TOP 10 RESTAURANTS LIKE Pai Vihar WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Burma Burma,"Asian, Burmese",4.74,1.5
Skoolroom,"Cafe, Continental, Italian, Burger, Beverages",4.23,700.0
Ilyazsab The House Of Chicken,"Rolls, Kebab",3.84,250.0
Royal Biryani Kitchen,"Biryani, North Indian, Kebab",3.72,600.0
Green Pepper,"Seafood, South Indian, Chinese, Kerala",3.65,600.0
Karavali Grand,"Mangalorean, Seafood, North Indian, Chinese",3.45,600.0
Cafe @ Elanza,"Chinese, North Indian, Cafe",3.45,1.0
Karavali Family Restaurant,"Seafood, Biryani, South Indian, North Indian, ...",3.19,500.0
Currylicious,"South Indian, Mangalorean, Kerala",2.81,800.0
Amaravati Authentic Andhra Restaurant,"Andhra, Chinese, North Indian, Biryani",2.57,600.0


In [31]:
recommend('The Grill House')

TOP 10 RESTAURANTS LIKE The Grill House WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
The Pizza Bakery,"Italian, Pizza, Beverages",4.87,1.2
Biergarten,"Continental, North Indian, Chinese, European, ...",4.83,2.1
Biergarten,"Continental, European, BBQ, Chinese, Asian",4.83,2.4
Hammered,"North Indian, South Indian, Continental, Asian",4.65,1.2
Hammered,"North Indian, Thai, Japanese, Continental, Cafe",4.65,1.3
Hammered,"North Indian, Continental, Cafe",4.65,1.2
Koramangala Social,"American, North Indian, Chinese, Finger Food",4.61,1.5
Ciclo Cafe,"Cafe, Italian, American",4.23,1.0
The Black Rabbit,"Continental, Finger Food",4.23,1.5
Cafe Azzure,"Cafe, Continental, Italian, Burger",4.15,1.2
